In [1]:
#-*- coding: windows-1250 -*-
import os
import glob
import pandas as pd

extension = 'csv'
os.chdir(r'c:\Users\marci\git\iMGW\dane_all')
result = glob.glob('*.{}'.format(extension))

lista_df =[]

for i in result:
    if "k_m_d" in str(i):
        df = pd.read_csv(i, header = None, encoding= 'windows-1250')
        lista_df.append(df)
        
df_polaczone = pd.concat(lista_df)

df_polaczone = df_polaczone.rename(columns={
                              0: "kod_stacji",1: "nazwa_stacji",2: "year",3: "month", 4: "t_max",5: "status_tmax",6: "sr_t_max",
                              7: "status_sr_tmax",8: "t_min",9: "status_tmin",10: "sr_t_min",11: "status_sr_min",12: "sr_temp_mies",
                              13: "stat_stm",14: "min_temp_grunt",15: "stat_tmng",16: "opady_mies_srenia",17: "status_opady_mies_srenia",
                              18: "opady_max_doba",19: "st_opady_max_doba",20: "pierw_dzien_opad_maks",21: "stat_opmx",
                              22: "pierw_dzien_opad_mak",23: "ostat_dzien_opad_maks",24: "maks_pokrywy", 25: "stat_pokrywy",
                              26: "liczba_dni_pokrywa",27: "liczba_dni_deszcz", 28: "liczba_dni_snieg"})

df_polaczone = df_polaczone[['nazwa_stacji', 'kod_stacji', 'year', 'month', 'sr_temp_mies', 'status_opady_mies_srenia']]

In [2]:
# zagreogowac rocznie
df_polaczone['data'] = df_polaczone['year'].astype(str) + "-" +df_polaczone['month'].astype(str) + "-01"
df_polaczone['data'] = pd.DatetimeIndex(df_polaczone['data'])
df_polaczone.set_index('data', inplace=True)

display(df_polaczone.tail(5))
lista_stacji = df_polaczone.kod_stacji.unique()

lista_df =[]
for stacja in lista_stacji:
    df_stacja = df_polaczone.loc[df_polaczone['kod_stacji'] == stacja]
    df_stacja_yearly = df_stacja.resample('Y').mean()
    lista_df.append(df_stacja_yearly)

df_polaczone_roczne = pd.concat(lista_df)
display(df_polaczone_roczne.head())

,nazwa_stacji,kod_stacji,year,month,sr_temp_mies,status_opady_mies_srenia
data,,,,,,
2019-07-01,OLECKO,254220090,2019,7,17.4,NaN
2019-08-01,OLECKO,254220090,2019,8,18.5,NaN
2019-09-01,OLECKO,254220090,2019,9,13.2,NaN
2019-10-01,OLECKO,254220090,2019,10,9.2,NaN
2019-11-01,OLECKO,254220090,2019,11,4.7,NaN


,kod_stacji,year,month,sr_temp_mies,status_opady_mies_srenia
data,,,,,
1952-12-31,249200180.0,1952.0,6.5,6.358333,NaN
1953-12-31,249200180.0,1953.0,6.5,7.208333,NaN
1954-12-31,249200180.0,1954.0,6.5,6.116667,NaN
1955-12-31,249200180.0,1955.0,6.5,6.158333,NaN
1956-12-31,249200180.0,1956.0,6.5,5.200000,NaN


In [3]:
# average reference period of 1961–1990 for each station
df_polaczone_roczne_avg = df_polaczone_roczne[(df_polaczone_roczne['year'] > 1961)]
df_polaczone_roczne_avg = df_polaczone_roczne_avg[(df_polaczone_roczne_avg['year'] < 1990)]

lista_df_avg =[]

for stacja in lista_stacji:
    df_stacja_avg = df_polaczone_roczne_avg.loc[df_polaczone_roczne_avg['kod_stacji'] == stacja]
    df_stacja_avg['avg_60_90'] = df_stacja_avg.sr_temp_mies.mean()
    df_stacja_avg= df_stacja_avg[['kod_stacji', 'avg_60_90']]
    lista_df_avg.append(df_stacja_avg)

    
# average over the standard reference period of 1961–1990.
df_avg_stacje = pd.concat(lista_df_avg)
df_avg_stacje = df_avg_stacje.drop_duplicates(subset=['kod_stacji', 'avg_60_90'])
display(df_avg_stacje.head(3))

C:\Users\marci\Anaconda2\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,kod_stacji,avg_60_90
data,,
1962-12-31,249200180.0,6.923810
1984-12-31,249209980.0,4.962500
1962-12-31,249220040.0,7.414583


In [4]:
# join
df_polaczone_roczne_avg = pd.merge(df_polaczone_roczne, df_avg_stacje, on='kod_stacji')

# monthly diffrence from the average over the standard reference period of 1961–1990.
df_polaczone_roczne_avg['anomalia'] = df_polaczone_roczne_avg['sr_temp_mies'] - df_polaczone_roczne_avg['avg_60_90']
df_polaczone_roczne_avg = df_polaczone_roczne_avg.drop_duplicates(subset=['kod_stacji', 'year'])
display(df_polaczone_roczne_avg.head())

,kod_stacji,year,month,sr_temp_mies,status_opady_mies_srenia,avg_60_90,anomalia
0,249200180.0,1952.0,6.5,6.358333,NaN,6.92381,-0.565476
1,249200180.0,1953.0,6.5,7.208333,NaN,6.92381,0.284524
2,249200180.0,1954.0,6.5,6.116667,NaN,6.92381,-0.807143
3,249200180.0,1955.0,6.5,6.158333,NaN,6.92381,-0.765476
4,249200180.0,1956.0,6.5,5.200000,NaN,6.92381,-1.723810


In [5]:
# GEOKODOWANIE - PROSTE
df_xy = pd.read_csv(r'c:\Users\marci\git\iMGW\csv\stacje_located.csv')

result = pd.merge(df_polaczone_roczne_avg,
                 df_xy[['kod_stacji', 'latitude', 'longitude']],
                 on='kod_stacji')

# DataFrame with x,y to GeoDataFrame
from shapely.geometry import Point
import geopandas as gpd

gdf = gpd.GeoDataFrame(
    result, geometry=gpd.points_from_xy(x=result.longitude, y=result.latitude),
    crs={'init':'epsg:4326'})

# reprojekcja do Panstwowego Ukladu Wspolrzednych Geodezyjnych PUWG1992
gdf= gdf.to_crs({'init': 'epsg:2180'})
liczba = gdf.shape[0]

gdf['year'] = gdf['year'].astype(str)
display(gdf.head())

,kod_stacji,year,month,sr_temp_mies,status_opady_mies_srenia,avg_60_90,anomalia,latitude,longitude,geometry
0,249200180.0,1952.0,6.5,6.358333,NaN,6.92381,-0.565476,49.706171,20.421727,POINT (602478.151 205279.732)
1,249200180.0,1953.0,6.5,7.208333,NaN,6.92381,0.284524,49.706171,20.421727,POINT (602478.151 205279.732)
2,249200180.0,1954.0,6.5,6.116667,NaN,6.92381,-0.807143,49.706171,20.421727,POINT (602478.151 205279.732)
3,249200180.0,1955.0,6.5,6.158333,NaN,6.92381,-0.765476,49.706171,20.421727,POINT (602478.151 205279.732)
4,249200180.0,1956.0,6.5,5.200000,NaN,6.92381,-1.723810,49.706171,20.421727,POINT (602478.151 205279.732)


In [10]:
# eksport shp co rok
lista_lat = range(1959,2020)

for rok in lista_lat:
        latacz = gpd.read_file(r'c:\Users\marci\git\iMGW\shp\latacz.shp')
        rok_format = "%.1f" % round(rok,1)
        gdf_rok = gdf.loc[gdf['year'] == rok_format]
        # outliers
        gdf_rok_outliers = gdf_rok.loc[gdf_rok['anomalia'] > -2.5]
        srednia = gdf_rok.anomalia.mean()
        latacz['anomalia'] = srednia
        gdf_rok = pd.concat([gdf_rok_outliers, latacz], sort=True)
        filename_shp = os.path.join(r"c:\Users\marci\git\iMGW\shp_output",  str(rok) + ".shp") 
        
        gdf_rok.to_file(driver = 'ESRI Shapefile', filename = filename_shp)
        

In [171]:
# selekcja unique stacji i geolokacja
df_lokacje = df_polaczone[['nazwa_stacji','kod_stacji']]
df_lokacje.drop_duplicates(inplace=True)
display(df_lokacje.head(5))

C:\Users\marci\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,nazwa_stacji,kod_stacji
data,,
1952-01-01,LIMANOWA,249200180
1955-01-01,KRYNICA-GÓRA PARKOWA,249209980
1951-01-01,DYNÓW,249220040
1951-01-01,SANOK-TREPCZA,249220080
1951-01-01,KARPACZ,250150220


In [56]:
# geocoding of stations
def geokoduj(df, pole):
    df['geo_location'] = df[pole].apply(geocode)
    located = df.loc[df["geo_location"].notnull()]
    not_located = df.loc[df["geo_location"].isnull()]
    return(located, not_located)

# geocoding
import geopy
import geopandas
from geopy.geocoders import Nominatim

locator = Nominatim(user_agent ='nlp2')
from geopy.extra.rate_limiter import RateLimiter

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

df_lokacje2 = df_lokacje.iloc[:10]
geokoduj = geokoduj(df_lokacje, 'nazwa_stacji')
located = geokoduj[0]

il_ogloszen = located.shape[0]
display('we have {} geocoded offers out of all {}'.format(il_ogloszen,df_lokacje.shape[0]))

# 3 - create longitude, laatitude and altitude from location column (returns tuple)
located['point'] = located['geo_location'].apply(lambda loc: tuple(loc.point) if loc else None)

# 4 - split point column into latitude, longitude and altitude columns
located[['latitude', 'longitude', 'altitude']] = pd.DataFrame(located['point'].tolist(), index=located.index)

# plotting of geocoded stations
# DataFrame with x,y to GeoDataFrame
from shapely.geometry import Point
import geopandas as gpd

gdf = gpd.GeoDataFrame(
    located, geometry=gpd.points_from_xy(x=located.longitude, y=located.latitude),
    crs={'init':'epsg:4326'})

# reprojekcja do Panstwowego Ukladu Wspolrzednych Geodezyjnych PUWG1992
gdf= gdf.to_crs({'init': 'epsg:2180'})
liczba = gdf.shape[0]

# plot geocoded offers
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['font.family'] = 'sans-serif'

pl_border = gpd.read_file(r'c:\Users\marci\git\iMGW\shp\pl_border.shp')
ax = pl_border.plot(color ='gainsboro', edgecolor='black')
ax.set_title("location of meteo stations, no: {}".format(liczba))
ax.legend()

gdf.plot(ax=ax, marker='o', color = 'red', markersize=5);

No handlers could be found for logger "geopy"
C:\Users\marci\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


'we have 328 geocoded offers out of all 338'

C:\Users\marci\Anaconda2\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\marci\Anaconda2\lib\site-packages\pandas\core\frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [60]:
located  = located[['latitude', 'longitude', 'kod_stacji']]
located.to_csv(r'c:\Users\marci\git\iMGW\csv\stacje_located2.csv')